In [25]:
import pandas as pd
import numpy as np
import re

In [3]:
%cd data
!ls 

/home/stephy/Learning_DataScience/Chemical_db/TCI_database_app/data
TCIAvailableStock.json		    TCI.smi
TCICompoundIdentifications.json     TCISpecifications.json
TCIGeneralInformation.json	    TCISpecificProperties.json
TCIGHSprecautionaryStatements.json  TCITransportationInformation.json
TCIRealtedLaws.json		    TCI.txt


In [4]:
df_stock = pd.read_json('TCIAvailableStock.json', orient ='split', compression = 'infer')

In [6]:
df_stock.sample(5).dropna(axis=1, how='all')

,code,OtherWH1G,price1G,Hyogo1G,Saitama1G,OtherWH25G,price25G,Hyogo25G,Saitama25G,OtherWH5G,price5G,Hyogo5G,Saitama5G,lowestPriceOption
22767,M3399,None,NaN,None,None,Contact Company,7000.0,Contact Company,20,None,NaN,None,None,7000.0
21665,M1976,None,NaN,None,None,Contact Company,24600.0,3,9,Contact Company,7000.0,Contact Company,6,7000.0
19650,I0996,None,NaN,None,None,Contact Company,13600.0,Contact Company,Contact Company,Contact Company,4100.0,Contact Company,Contact Company,4100.0
12822,D3271,None,NaN,None,None,7,43500.0,9,9,Contact Company,11200.0,Contact Company,4,11200.0
4411,B2233,Contact Company,33600.0,20,3,None,NaN,None,None,None,NaN,None,None,33600.0


In [31]:
df_stock.head(n=5).dropna(axis=1, how='all')

,code,OtherWH100G,price100G,Hyogo100G,Saitama100G,OtherWH25G,price25G,Hyogo25G,Saitama25G,OtherWH500G,price500G,Hyogo500G,Saitama500G,OtherWH5G,price5G,Hyogo5G,Saitama5G,lowestPriceOption
0,A0001,None,NaN,None,None,20,6200.0,20,18,Contact Company,43800.0,Contact Company,14,None,NaN,None,None,6200.0
1,A0002,None,NaN,None,None,20,14800.0,1,20,None,NaN,None,None,None,NaN,None,None,14800.0
2,A0003,Contact Company,8200.0,Contact Company,Contact Company,Contact Company,2800.0,11,20,Contact Company,26100.0,11,12,None,NaN,None,None,2800.0
3,A0004,20,48800.0,20,6,20,14800.0,10,20,None,NaN,None,None,None,NaN,None,None,14800.0
4,A0005,None,NaN,None,None,Contact Company,10500.0,20,20,None,NaN,None,None,Contact Company,3100.0,8,20,3100.0


In [119]:
column_names = ['code', 'unit', 'amount', 'price', 'stock_at_hyogo', 'stock_at_saitama',\
                'stock_at_other']

availability_list = []

for index, row in df_stock.head(n=5).dropna(axis=1, how='all').iterrows(): 
    for key in row.keys():
        if 'price' in key:
            amount = re.findall(r'\d+', key)[0]
            unit = key.split(amount)[1]
            price = row[key]
            hyogo = row['Hyogo{0}{1}'.format(amount, unit)]
            saitama = row['Saitama{0}{1}'.format(amount, unit)]
            other = row['OtherWH{0}{1}'.format(amount, unit)]
            
            #get the amount of good in Hyogo, Saitama and Other Warehouses
            try:
                stock_at_hyogo = hyogo.isdigit()
            except AttributeError:
                stock_at_hyogo = False
                
            try:
                stock_at_saitama = saitama.isdigit()
            except AttributeError:
                stock_at_saitama = False
                
            try:
                stock_at_other = other.isdigit()
            except AttributeError:
                stock_at_other = False
            
            dictionary = {'code': row['code'], 'unit':unit, 'amount':amount, 'price':price,\
                          'stock_at_hyogo':stock_at_hyogo, 'stock_at_saitama':stock_at_saitama,\
                          'stock_at_other':stock_at_other}
            
            availability_list.append(dictionary)

[{'code': 'A0001',
  'unit': 'G',
  'amount': '100',
  'price': nan,
  'stock_at_hyogo': False,
  'stock_at_saitama': False,
  'stock_at_other': False},
 {'code': 'A0001',
  'unit': 'G',
  'amount': '25',
  'price': 6200.0,
  'stock_at_hyogo': True,
  'stock_at_saitama': True,
  'stock_at_other': True},
 {'code': 'A0001',
  'unit': 'G',
  'amount': '500',
  'price': 43800.0,
  'stock_at_hyogo': False,
  'stock_at_saitama': True,
  'stock_at_other': False},
 {'code': 'A0001',
  'unit': 'G',
  'amount': '5',
  'price': nan,
  'stock_at_hyogo': False,
  'stock_at_saitama': False,
  'stock_at_other': False},
 {'code': 'A0002',
  'unit': 'G',
  'amount': '100',
  'price': nan,
  'stock_at_hyogo': False,
  'stock_at_saitama': False,
  'stock_at_other': False},
 {'code': 'A0002',
  'unit': 'G',
  'amount': '25',
  'price': 14800.0,
  'stock_at_hyogo': True,
  'stock_at_saitama': True,
  'stock_at_other': True},
 {'code': 'A0002',
  'unit': 'G',
  'amount': '500',
  'price': nan,
  'stock_at_h

In [122]:
availability_df = pd.DataFrame(availability_list)
availability_df

,code,unit,amount,price,stock_at_hyogo,stock_at_saitama,stock_at_other
0,A0001,G,100,NaN,False,False,False
1,A0001,G,25,6200.0,True,True,True
2,A0001,G,500,43800.0,False,True,False
3,A0001,G,5,NaN,False,False,False
4,A0002,G,100,NaN,False,False,False
5,A0002,G,25,14800.0,True,True,True
6,A0002,G,500,NaN,False,False,False
7,A0002,G,5,NaN,False,False,False
8,A0003,G,100,8200.0,False,False,False
9,A0003,G,25,2800.0,True,True,False


In [123]:
availability_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   code              20 non-null     object 
 1   unit              20 non-null     object 
 2   amount            20 non-null     object 
 3   price             10 non-null     float64
 4   stock_at_hyogo    20 non-null     bool   
 5   stock_at_saitama  20 non-null     bool   
 6   stock_at_other    20 non-null     bool   
dtypes: bool(3), float64(1), object(3)
memory usage: 828.0+ bytes
